In [1]:
import tensorflow as tf

In [2]:
tf.__version__

'1.2.1-rc1'

In [4]:
import matplotlib.pyplot as plt
import numpy as np

In [3]:
node1 = tf.constant(3.0, dtype=tf.float32)
node2 = tf.constant(4.0) # also tf.float32 implicitly
print(node1, node2)

Tensor("Const:0", shape=(), dtype=float32) Tensor("Const_1:0", shape=(), dtype=float32)


In [4]:
print(tf.constant(4.0)+tf.constant(4.0))

Tensor("add:0", shape=(), dtype=float32)


In [5]:
sess = tf.Session()
print(sess.run([node1, node2]))

[3.0, 4.0]


In [6]:
node3 = tf.add(node1, node2)
print("node3:", node3)
print("sess.run(node3):", sess.run(node3))

node3: Tensor("Add:0", shape=(), dtype=float32)
sess.run(node3): 7.0


In [7]:
print("sess.run(node3):", sess.run(node1 + node2))

sess.run(node3): 7.0


In [8]:
a = tf.placeholder(tf.float32)
b = tf.placeholder(tf.float32)
adder_node = a + b  # + provides a shortcut for tf.add(a, b)

In [9]:
sess.run(adder_node, {a:3, b:4.5})

7.5

In [10]:
add_and_triple = adder_node * 3.
print(sess.run(add_and_triple, {a: 3, b: 4.5}))

22.5


In [11]:
W = tf.Variable([.3], dtype=tf.float32)
b = tf.Variable([-.3], dtype=tf.float32)
x = tf.placeholder(tf.float32)
linear_model = W * x + b

#### To initialize all the variables in a TensorFlow program

In [12]:
init = tf.global_variables_initializer()
sess.run(init)

### It is important to realize `init` is a handle to the TensorFlow sub-graph that initializes all the global variables. Until we call `sess.run`, the variables are uninitialized.

In [13]:
print(sess.run(linear_model, {x: [1, 2, 3, 4]}))

[ 0.          0.30000001  0.60000002  0.90000004]


In [14]:
y=sess.run(linear_model, {x: [1, 2, 3, 4]})

### Calculo do erro quadrado médio

In [15]:
y = tf.placeholder(tf.float32)
squared_deltas = tf.square(linear_model - y)
loss = tf.reduce_sum(squared_deltas)
print(sess.run(loss, {x: [1, 2, 3, 4], y: [0, -1, -2, -3]}))

23.66


### Para mudar o valor inicial das variáveis inicializadas `tf.assign`

In [16]:
fixW = tf.assign(W, [-1.])
fixb = tf.assign(b, [1.])
sess.run([fixW, fixb])
print(sess.run(loss, {x: [1, 2, 3, 4], y: [0, -1, -2, -3]}))

0.0


In [17]:
optimizer = tf.train.GradientDescentOptimizer(0.01)
train = optimizer.minimize(loss)

In [18]:
sess.run(init) # reset values to incorrect defaults.

In [20]:
print(sess.run(b))

[-0.30000001]


In [22]:
sess.run(init) # reset values to incorrect defaults.
for i in range(1000):
  sess.run(train, {x: [1, 2, 3, 4], y: [0, -1, -2, -3]})

print(sess.run([W, b]))

[array([-0.99999911], dtype=float32), array([ 0.99999744], dtype=float32)]


In [23]:
# training data
x_train = [1, 2, 3, 4]
y_train = [0, -1, -2, -3]

# evaluate training accuracy
curr_W, curr_b, curr_loss = sess.run([W, b, loss], {x: x_train, y: y_train})
print("W: %s b: %s loss: %s"%(curr_W, curr_b, curr_loss))


W: [-0.99999911] b: [ 0.99999744] loss: 4.20641e-12


### tf.estimator

In [10]:
np.shape(feature_columns)

(1, 5)

In [5]:
# Declare list of features. We only have one numeric feature. There are many
# other types of columns that are more complicated and useful.
feature_columns = [tf.feature_column.numeric_column("x", shape=[1])]

# An estimator is the front end to invoke training (fitting) and evaluation
# (inference). There are many predefined types like linear regression,
# linear classification, and many neural network classifiers and regressors.
# The following code provides an estimator that does linear regression.
estimator = tf.estimator.LinearRegressor(feature_columns=feature_columns)

# TensorFlow provides many helper methods to read and set up data sets.
# Here we use two data sets: one for training and one for evaluation
# We have to tell the function how many batches
# of data (num_epochs) we want and how big each batch should be.
x_train = np.array([1., 2., 3., 4.])
y_train = np.array([0., -1., -2., -3.])
x_eval = np.array([2., 5., 8., 1.])
y_eval = np.array([-1.01, -4.1, -7, 0.])
input_fn = tf.estimator.inputs.numpy_input_fn(
    {"x": x_train}, y_train, batch_size=4, num_epochs=None, shuffle=True)
train_input_fn = tf.estimator.inputs.numpy_input_fn(
    {"x": x_train}, y_train, batch_size=4, num_epochs=1000, shuffle=False)
eval_input_fn = tf.estimator.inputs.numpy_input_fn(
    {"x": x_eval}, y_eval, batch_size=4, num_epochs=1000, shuffle=False)

# We can invoke 1000 training steps by invoking the  method and passing the
# training data set.
estimator.train(input_fn=input_fn, steps=1000)

# Here we evaluate how well our model did.
train_metrics = estimator.evaluate(input_fn=train_input_fn)
eval_metrics = estimator.evaluate(input_fn=eval_input_fn)
print("train metrics: %r"% train_metrics)
print("eval metrics: %r"% eval_metrics)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': '/tmp/tmp9fcm2zyx', '_tf_random_seed': 1, '_save_summary_steps': 100, '_save_checkpoints_secs': 600, '_save_checkpoints_steps': None, '_session_config': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100}
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Saving checkpoints for 1 into /tmp/tmp9fcm2zyx/model.ckpt.
INFO:tensorflow:loss = 14.0, step = 1
INFO:tensorflow:global_step/sec: 955.259
INFO:tensorflow:loss = 0.112379, step = 101 (0.106 sec)
INFO:tensorflow:global_step/sec: 1000.55
INFO:tensorflow:loss = 0.0228326, step = 201 (0.100 sec)
INFO:tensorflow:global_step/sec: 984.967
INFO:tensorflow:loss = 0.0105294, step = 301 (0.102 sec)
INFO:tensorflow:global_step/sec: 1035.54
INFO:tensorflow:loss = 0.0023707, step = 401 (0.097 sec)
INFO:tensorflow:global_step/sec: 1068.08
INFO:tensorflow:loss = 0.000313843, step = 501 (0.097 sec)
INFO:tensorf